### Wstęp do Uczenia Maszynowego 
##### Laboratorium 8

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split


### 1. Naive Bayes

In [2]:
X = pd.read_csv("X.csv")
y = pd.read_csv("y.csv")

### Zadanie 1
-----
Przygotuj dane w następujący sposób:

a) Ze zbioru `X` wybierz tylko zmienne `"education", "workclass", "sex", "hours-per-week"`.

b) Dokonaj transformacji dla zmiennej `"education"`

    1, education in ['Masters', 'some-college', 'Bachelors', 'Doctorate']
    0, w przeciwnym przypadku

c) Dokonaj transformacji dla zmiennej `"workclass"`

    1, workclass == 'Private'
    0, w przeciwnym przypadku

d) Dokonaj transformacji dla zmiennej `"sex"`

    1, sex == 'Male
    0, w przeciwnym przypadku

e) Dokonaj transformacji dla zmiennej `"hours-per-week"`

    1, hours-per-week <= 40 
    0, w przeciwnym przypadku

In [3]:
tmp = X[["education", "workclass", "sex", "hours-per-week"]]
tmp

,education,workclass,sex,hours-per-week
0,Bachelors,State-gov,Male,40
1,Bachelors,Self-emp-not-inc,Male,13
2,HS-grad,Private,Male,40
3,11th,Private,Male,40
4,Bachelors,Private,Female,40
...,...,...,...,...
48837,Bachelors,Private,Female,36
48838,HS-grad,NaN,Male,40
48839,Bachelors,Private,Male,50
48840,Bachelors,Private,Male,40


In [4]:
X = pd.DataFrame({"education" : tmp["education"].isin(["some-college", "Masters", "Bachelors", "Doctorate"]),
                  "workclass" : tmp["workclass"].isin(["Private"]) ,
                  "sex" : tmp["sex"].isin(["Male"]),
                  "hours" : np.where(tmp["hours-per-week"] <= 40, True, False)})

In [5]:
X

,education,workclass,sex,hours
0,True,False,True,True
1,True,False,True,True
2,False,True,True,True
3,False,True,True,True
4,True,True,False,True
...,...,...,...,...
48837,True,True,False,True
48838,False,False,True,True
48839,True,True,True,False
48840,True,True,True,True


In [6]:
y.value_counts()

income
<=50K     24720
<=50K.    12435
>50K       7841
>50K.      3846
dtype: int64

In [7]:
y.loc[y.income == '<=50K.'] = '<=50K'
y.loc[y.income == '>50K.'] = '>50K'

In [8]:
y.value_counts()

income
<=50K     37155
>50K      11687
dtype: int64

### Zadanie 2
-----
Wylicz prawdopodobieństwa dla poszczególnych zmiennych pod warunkiem `Y`. Oblicz przynależność do klasy dla pierwszej obserwacji ze zbioru testowego na postawie wyliczonych prawdopodobieństw.

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [10]:
print('<=50K:', np.mean(y_train.income == "<=50K"))
print('>50K:', np.mean(y_train.income != "<=50K"))

<=50K: 0.7586312799119597
>50K: 0.24136872008804033


In [11]:
all = pd.concat([X_train, y_train], axis=1)

In [12]:
print(round(all.groupby(['income'])['education'].value_counts(normalize = True).unstack(), 4))
print(round(all.groupby(['income'])['workclass'].value_counts(normalize = True).unstack(), 4))
print(round(all.groupby(['income'])['sex'].value_counts(normalize = True).unstack(), 4))
print(round(all.groupby(['income'])['hours'].value_counts(normalize = True).unstack(), 4))

education   False   True 
income                   
<=50K      0.8370  0.1630
>50K       0.5547  0.4453
workclass   False   True 
income                   
<=50K      0.2875  0.7125
>50K       0.3732  0.6268
sex      False   True 
income                
<=50K   0.3887  0.6113
>50K    0.1522  0.8478
hours    False   True 
income                
<=50K   0.2310  0.7690
>50K    0.4941  0.5059


In [13]:
X_test.iloc[0,:]

education    False
workclass     True
sex          False
hours         True
Name: 2808, dtype: bool

In [14]:
# dla y = "<=50K"
0.1633 * 0.2860 * 0.3882 * 0.7689 * 0.7606019501957874

0.010603153396946147

In [15]:
# dla y = ">50K"
0.4451 * 0.3671 * 0.1500 * 0.5106 * 0.23939804980421261

0.0029959506585458145

Wniosek: predykcja dla obserwacji to "<=50K"

### Zadanie 3
-----
Przygotuj model Naive Bayes używająć funkcji `BernouliiNB`. 
Jakie są dostępne inne warianty modelu Naive Bayes i czym się charakteryzują?

In [16]:
from sklearn.naive_bayes import BernoulliNB

In [17]:
NB = BernoulliNB()
NB.fit(X_train, y_train.income)

BernoulliNB()

In [18]:
pred = NB.predict(X_test)

In [19]:
pred[0]
# zgadza się z wyliczeniami z Zadania 2

'<=50K'

### 2. kNN

### Zadanie 4
-----

a) Podziel losowo zbiór na część treningową i testową (7:3).

b) Przeskaluj zmienne w zbiorze treningowym i testowym za pomocą średniej i odchylenia standardowego danej zmiennej na zbiorze treningowym.

c) Dopasuj metodę k najbliższych sąsiadów dla $k = 5, 10, 20, 50, 80, 100, 150$ na zbiorze treningowym. Rozważ przynajmniej 3-krotną kroswalidację.

d) Sprawdź na zbiorze testowym współczynnik AUC dla wyliczonych $k$ z polecenia c).

e) Wybierz optymalne $k$ bazując na punkcie c) i d). Przygotuj model kNN z jednakową wagą dla wszystkich sąsiadów oraz z wagą uzależnioną od odległości między obserwacją a jej sąsiadem.

In [20]:
df = pd.read_csv("SAheart.data")

In [21]:
y = df.chd
X = df.drop(['chd'], axis = 1)
X = pd.get_dummies(X, drop_first=True)


In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

In [23]:
X_train

,row.names,sbp,tobacco,ldl,adiposity,typea,obesity,alcohol,age,famhist_Present
364,366,156,3.02,5.35,25.72,53,25.22,28.11,52,1
226,227,112,4.46,7.18,26.25,69,27.29,0.00,32,1
255,256,164,13.02,6.26,29.38,47,22.75,37.03,54,1
224,225,132,7.28,3.52,12.33,60,19.48,2.06,56,0
109,110,124,0.00,3.04,17.33,49,22.04,0.00,18,0
...,...,...,...,...,...,...,...,...,...,...
420,422,120,5.50,3.51,23.23,46,22.40,90.31,43,0
101,102,166,0.07,4.03,29.29,53,28.37,0.00,27,0
438,440,130,1.22,3.30,13.65,50,21.40,3.81,31,0
84,85,128,0.00,2.43,13.15,63,20.75,0.00,17,1


In [24]:
from sklearn.preprocessing import StandardScaler

In [25]:
scaler = StandardScaler() 
X_train_std = scaler.fit_transform(X_train) 
X_test_std = scaler.transform(X_test)

In [26]:
import sklearn.model_selection as skm
kfold = skm.KFold(5,
                  random_state=0,
                  shuffle=True)

In [27]:
from sklearn.neighbors import KNeighborsClassifier

In [28]:
knn = KNeighborsClassifier()

In [29]:
grid = skm.GridSearchCV(knn,
                        {'n_neighbors':[5, 10, 20, 50, 80, 100, 150]}, 
                         refit=True,
                         cv=kfold,
                         scoring='roc_auc')

In [30]:
grid.fit(X_train_std, y_train)

GridSearchCV(cv=KFold(n_splits=5, random_state=0, shuffle=True),
             estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [5, 10, 20, 50, 80, 100, 150]},
             scoring='roc_auc')

In [31]:
grid.best_params_

{'n_neighbors': 50}

In [32]:
knn = KNeighborsClassifier(n_neighbors = grid.best_params_["n_neighbors"])
knn_distance = KNeighborsClassifier(n_neighbors = grid.best_params_["n_neighbors"], weights = "distance")

knn.fit(X_train_std, y_train)
knn_distance.fit(X_train_std, y_train)

from sklearn.metrics import roc_auc_score

print("knn: ", roc_auc_score(y_test, knn.predict_proba(X_test_std)[:,1]))
print("knn_distance :", roc_auc_score(y_test, knn_distance.predict_proba(X_test_std)[:,1]))

knn:  0.7688172043010751
knn_distance : 0.7660121552127162
